In [1]:
import py_vollib.black_scholes.implied_volatility as iv
import py_vollib.black_scholes as bs
import py_vollib.black_scholes.greeks.numerical as greek
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def get_vol(option_price, spot, strike, T, r=0, option_type='p'):
    return iv.implied_volatility(option_price, spot, strike, T, r, option_type)

def get_greeks(option_price, spot, strike, T, r=0, option_type='p'):
    iv = get_vol(option_price, spot, strike, T, r)
    gamma = greek.agamma(option_type, spot, strike, T, r, iv)
    theta = greek.atheta(option_type, spot, strike, T, r, iv)
    delta = greek.adelta(option_type, spot, strike, T, r, iv)
    vega = greek.avega(option_type, spot, strike, T, r, iv)
    return [iv, gamma, theta, vega, delta]

In [3]:
spot = 209.10
T1 = (200/24)/365 # (hours until expiry /24) / 365 == Years Until Maturity
strike_one = 200
price_one = 6.0824

In [37]:
T1

0.022831050228310504

In [8]:
one = get_greeks(price_one, spot, strike_one, T1)
df = pd.DataFrame([one])
df.columns = ['implied_vol', 'gamma', 'theta', 'vega', 'delta']
df

,implied_vol,gamma,theta,vega,delta
0,0.809967,0.014245,-0.559716,0.115173,-0.335506


### Corrado Miller Put Option Implied Volatility Approximation

In [48]:
# Closed form solution to calculate IV - useful for when we program on ETH Blockchain in Solidity
def put_corrado_miller(price, strike, spot, time):
    fact_1 = np.sqrt((2*np.pi)/time)*(1/(spot + strike))
    fact_2 = (price - (strike-spot)/2)
    fact_3 = (price - (strike-spot)/2)**2
    fact_4 = ((spot - strike)**2)/(np.pi)
    iv = fact_1*(fact_2 + np.sqrt(fact_3 - fact_4))
    return iv

In [49]:
# We get a similar IV as the iterative solution
put_corrado_miller(9.45, 200, 199.21 , (7 + 5/24)/365)

0.8086725044084013

### Put Option Approximated Value 

In [158]:
# Closed form solution to calculate IV - useful for when we program on ETH Blockchain in Solidity
def put_corrado_miller(price, strike, spot, time):
    fact_1 = np.sqrt((2*np.pi)/time)*(1/(spot + strike))
    fact_2 = (price - (strike-spot)/2)
    fact_3 = (price - (strike-spot)/2)**2
    fact_4 = ((spot - strike)**2)/(np.pi)
    iv = fact_1*(fact_2 + np.sqrt(fact_3 - fact_4))
    return iv

# Approach 1 to calculate put price
def method_one_price(iv, spot, strike, time):
    # https://www.youtube.com/watch?v=O7u0ql9iwJE
    # taylor series approximation
    return (0.40*(spot - ((strike-spot)/2))*(iv*np.sqrt(time)) + (strike-spot)/2)

# Approach 2 to calculate put price - not as accurate
def method_two_price(iv, spot, strike, time):
    #http://www.espenhaug.com/black_scholes.html
    # Brenner and Subrahmanyam (1988) approximation
    # only good if option is ATM and short time maturity
    return (spot*0.40*iv*np.sqrt(time))

# Analyzing the effectiveness of these models on the May29-ETH-200 Put on Deribit
time = (7 + 4/24)/365
iv_aprx = put_corrado_miller(10.44, 200, 197.26 , time)

# decently close
method_one_price(iv_aprx, 197.26, 200, time)

# not as good
method_two_price(iv_aprx, 197.26, 200, time)

def put_delta_arpx(time, iv):
    # works best for ATM options
    return  -(0.5 + (iv*np.sqrt(time)/(2*np.sqrt(2*np.pi))))

def vega_aprx(spot, time):
    # works best for ATM options
    return 0.4*spot*np.sqrt(time)/100

def gamma_aprx(spot, time, iv):
    # https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781119206965.app2
    # works best for ATM options
    return 0.4/(spot*iv*np.sqrt(time))


(put_delta_arpx(time, iv_aprx), 
 vega_aprx(spot, time), 
 gamma_aprx(spot, time, iv_aprx))

(-0.5226643636817433, 0.11719961755980426, 0.016836138828754964)